In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from google.colab import drive
from array import array
import pandas as pd
import os
import seaborn as sns
drive.mount('/content/gdrive') # 此處需要登入google帳號
data = pd.read_csv("/content/gdrive/My Drive/thesis/thesisdataset/millan_average13_629_final.csv")#讀取資料集1
data.shape

In [ ]:
data2 = pd.read_csv("/content/gdrive/My Drive/thesis/thesisdataset/taiwan_average37_83_final.csv")#讀取資料集2
data2.shape

In [ ]:
data3 = pd.read_csv("/content/gdrive/My Drive/thesis/thesisdataset/finland_average_47_786_final.csv")#讀取資料集3
data3.shape

In [ ]:
!pip install learn2learn

In [ ]:
!nvidia-smi

In [ ]:
df = pd.DataFrame(data)
dataset = df.internet.values.astype(float)

In [ ]:
df2 = pd.DataFrame(data2)
dataset2 = df2.load.values.astype(float)

In [ ]:
df4 = pd.DataFrame(data3)
dataset4 = df4.Internet.values.astype(float)

In [ ]:
dataset1000=np.append(dataset2, dataset4)

In [ ]:
dataset1000=np.append(dataset1000, dataset) #聚合三個資料集

In [ ]:

dataset1000.shape

In [ ]:

dataset1000=dataset1000.reshape(-1,1)#reshape以利放入模型

In [ ]:
dataset1000.shape

In [ ]:
#訓練集測試集產生
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

sc = MinMaxScaler()
dataset1000 = sc.fit_transform(dataset1000)

seq_length = 1
x, y = sliding_windows(dataset1000, seq_length)
train_size = 1516090
#test_size = 898761
#train_size = int(len(y) * 0.6)
test_size = len(y) - train_size

dataX = Variable(torch.Tensor(np.array(x)))
dataY = Variable(torch.Tensor(np.array(y)))

trainX = Variable(torch.Tensor(np.array(x[0:train_size])))
trainY = Variable(torch.Tensor(np.array(y[0:train_size])))

testX = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])))

In [ ]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

In [ ]:
#形成元窗格
trainX=trainX.reshape(10,151609,1,1)
trainY=trainY.reshape(10,151609,1)

In [ ]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

In [ ]:
#LSTM宣告
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out

In [ ]:
import random
torch.manual_seed(0)

seed=0

random.seed(seed)
np.random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Remove randomness (may be slower on Tesla GPUs) 
# https://pytorch.org/docs/stable/notes/randomness.html
if seed ==0 :
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
#訓練過程
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import learn2learn as l2l
epochs = 1000
learning_rate = 0.01
input_size = 1
hidden_size = 2
num_layers = 1
num_classes = 1
meta_learning_rate = 0.01
adaptation_steps = 1
encoder = LSTM(num_classes, input_size, hidden_size, num_layers)
output_layer = nn.Linear(1, 1)
maml = l2l.algorithms.MAML(output_layer, lr=learning_rate, first_order=False)
opt = optim.Adam(list(maml.parameters()) + list(encoder.parameters()), lr=meta_learning_rate)
loss_fn = torch.nn.MSELoss()
for iteration in range(epochs):
    opt.zero_grad()
    iteration_error = 0.0
    for task in range(9):
        learner = maml.clone()
        x_spt = trainX[task]
        y_spt = trainY[task]
        x_qry = trainX[task+1]
        y_qry = trainY[task+1]
        # Fast adapt
        for _ in range(adaptation_steps):
            pred = learner(encoder(x_spt))
            error = loss_fn(pred, y_spt)
            learner.adapt(error)
            print("Epoch: %d, Task: %d, loss: %1.5f" % (iteration, task, error.item()))

        pred = learner(encoder(x_qry))
        evaluation_error = loss_fn(pred, y_qry)
        print(evaluation_error)
        iteration_error += evaluation_error
    # Meta-update the model parameters
    iteration_error /= 10
    print(iteration_error)
    iteration_error.backward()   
    opt.step()


In [ ]:
testXall=testX.reshape(-1,1,1)
testYall=testY.reshape(-1,1)

In [ ]:
#結果測試
learner.eval()
test_predict = learner(encoder(testXall))
test_predict = test_predict.data.numpy()
testY_plot = testYall.data.numpy()
#data_predict=data_predict.reshape(1, -1)
test_predict = sc.inverse_transform(test_predict)
testY_plot = sc.inverse_transform(testY_plot)

#plt.axvline(x=train_size, c='r', linestyle='--')


plt.plot(testY_plot,color='blue',label='True Value')
plt.plot(test_predict,color='red',label='Predicted Value')
plt.suptitle('MAML-LSTM Internet Traffic Prediction')
plt.show()
import math
testScore = math.sqrt(mean_squared_error(testY_plot, test_predict))
print('Train Score: %.2f RMSE' % (testScore))
#testScore = math.sqrt(mean_squared_error(testY, test_predict))
#print('Test Score: %.2f RMSE' % (testScore))

from math import sqrt

from sklearn.metrics import mean_absolute_error

from sklearn.metrics import mean_squared_error

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error


print("mean_absolute_error:",mean_absolute_error(testY_plot, test_predict))

print("mean_squared_error:",mean_squared_error(testY_plot,test_predict))

print("rmse:",sqrt(mean_squared_error(testY_plot, test_predict)))

print("r2 score:",r2_score(testY_plot, test_predict))
print("mape_sk", mean_absolute_percentage_error(testY_plot, test_predict))

In [ ]:
#儲存模型
import joblib
joblib.dump(learner,"/content/gdrive/My Drive/re_thesis/re_thesis_test/model_test_2/lstmlearner.pkl")
joblib.dump(encoder,"/content/gdrive/My Drive/re_thesis/re_thesis_test/model_test_2/lstmencoder.pkl")

['/content/gdrive/My Drive/re_thesis/re_thesis_test/model_test_2/lstmencoder.pkl']